In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
avg = None

def running_avg(frame, alpha=0.01):
    global avg
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    if avg is None:
        avg = gray.copy().astype("float")
        return np.zeros_like(gray)

    cv2.accumulateWeighted(gray, avg, alpha)
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
    _, thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)
    return thresh

fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

def mog2_method(frame):
    fgmask = fgbg.apply(frame)
    return fgmask

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (640, 360))

    ra_mask = running_avg(frame_resized)
    mog2_mask = mog2_method(frame_resized)

    cv2.imshow("Original", frame_resized)
    cv2.imshow("Running Average", ra_mask)
    cv2.imshow("MOG2", mog2_mask)

    key = cv2.waitKey(30) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [8]:
import cv2
import numpy as np

# Open a video capture
cap = cv2.VideoCapture("sample.mp4")  # Use 0 for webcam, or replace with video path

# --- Running Average Background Subtraction ---
avg = None  # Initialize average frame

# --- MOG2 Background Subtraction ---
mog2 = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize for faster processing (optional)
    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # --- Running Average ---
    if avg is None:
        avg = gray.copy().astype("float")
        continue

    cv2.accumulateWeighted(gray, avg, 0.05)  # 0.05 is the learning rate
    running_avg = cv2.convertScaleAbs(avg)
    diff = cv2.absdiff(gray, running_avg)
    _, running_avg_mask = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

    # --- MOG2 ---
    mog2_mask = mog2.apply(frame)

    # Display results
    cv2.imshow("Original", frame)
    cv2.imshow("Running Avg Mask", running_avg_mask)
    cv2.imshow("MOG2 Mask", mog2_mask)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()